<a href="https://colab.research.google.com/github/Corrado777/CodeAstro9/blob/develop/Time_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import datetime
from astropy.time import Time
from astropy.coordinates import get_sun, AltAz, solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body
import astropy.units as u

class TimeConverter:
    def __init__(self, earth_date, earth_time):
        self.earth_date = earth_date
        self.earth_time = earth_time

        # Constants based on Earth and Mars day lengths
        self.earth_day_seconds = 24 * 60 * 60  # seconds in an Earth day
        self.mars_day_seconds = 24.6597 * 60 * 60  # seconds in a Mars day

    def convert_to_mars_time(self):
        # Extract Earth time components
        earth_hours, earth_minutes, earth_seconds = self.earth_time

        # Convert Earth date and time to total seconds since the beginning of epoch (for simplicity)
        earth_total_seconds = (self.earth_date - datetime.datetime(1609, 3, 12)).total_seconds() \
                              + earth_hours * 3600 + earth_minutes * 60 + earth_seconds

        # Calculate Mars time in sols (Martian days)
        mars_sols = earth_total_seconds / self.mars_day_seconds

        # Calculate Mars time in years
        mars_years = mars_sols / 669.6

        # Convert remaining seconds to Martian time
        mars_seconds = earth_total_seconds % self.mars_day_seconds
        mars_hours = mars_seconds // 3600
        mars_seconds %= 3600
        mars_minutes = mars_seconds // 60
        mars_seconds %= 60

        return mars_years, mars_sols, int(mars_hours), int(mars_minutes), int(mars_seconds)

    def get_sun_position_on_mars(self, mars_location):
        # Create a Time object for the given Earth date and time
        time = Time(self.earth_date + datetime.timedelta(hours=self.earth_time[0], minutes=self.earth_time[1], seconds=self.earth_time[2]))

        # Use the solar system ephemeris to calculate the Sun's position
        with solar_system_ephemeris.set('builtin'):
            mars = get_body('mars', time)
            sun = get_sun(time)

        # Transform Sun's position to the local AltAz frame on Mars
        altaz = AltAz(obstime=time, location=mars_location)
        sun_altaz = sun.transform_to(altaz)

        return sun_altaz.alt, sun_altaz.az

# Example usage
earth_date = datetime.datetime(2024, 5, 25)  # Year, Month, Day
earth_time = (1, 30, 45)  # Hours, Minutes, Seconds
mars_location = EarthLocation(lat=4.5*u.deg, lon=137.4*u.deg, height=0*u.m)  # Latitude, Longitude, Elevation on Mars (e.g., Hellas Time)

converter = TimeConverter(earth_date, earth_time)
mars_years, mars_sols, mars_hours, mars_minutes, mars_seconds = converter.convert_to_mars_time()
sun_alt, sun_az = converter.get_sun_position_on_mars(mars_location)

print(f"Earth date and time: {earth_date.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mars time: {int(mars_years)} years, {int(mars_sols)} sols, {mars_hours} hours, {mars_minutes} minutes, {mars_seconds} seconds")
print(f"Sun's position on Mars at the given time: Altitude = {sun_alt:.2f}, Azimuth = {sun_az:.2f}")


Earth date and time: 2024-05-25 00:00:00
Mars time: 220 years, 147593 sols, 2 hours, 24 minutes, 37 seconds
Sun's position on Mars at the given time: Altitude = 65.13 deg, Azimuth = 46.73 deg
